In [17]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

# The Task: CIFAR Classification
<img src="cifar.png">

In this lab, we'll build a neural network to classify images on CIFAR 10.



## Step 1: Loading Data and Preprocessing
Let's start by loading the data.
We're going to normalize our images to have 0 mean, and unit variance. We'll do this using some torchvision transforms. This generally helps stablize learning, and is common practice. 

In [18]:
normalize_image = transforms.Compose([
                           transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))   
                       ])

all_train = datasets.CIFAR10('data', train=True, download=True,
                        transform=normalize_image)
num_train = int(len(all_train)*.8)
train = [all_train[i] for i in range(num_train)]
dev = [all_train[i] for i in range(num_train,len(all_train))]
test = datasets.CIFAR10('data', train=False, download=True, 
                      transform=normalize_image)
                           


Files already downloaded and verified
Files already downloaded and verified


In [19]:
train[0][0].size()
print(train[0][0])

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

## Step 2: Building a model

All pytorch models should be implemented as instances of `nn.Module`. 

To build a model you need to:
a) define what parameters it'll need in it's `__init__` function
b) define the model's computation, using those parameters, in a forward function.


Experiment with different neural architectures. I recommend starting with a simple linear model and building up from there.

In [20]:
class Model(nn.Module):
#    def __init__(self):
#        super(Model, self).__init__()
#        self.fc = nn.Linear(32*32*3, 10)


#    def forward(self, x):
#        batch_size, num_channels, height, width = x.size()
#        x = x.view(batch_size, -1)
#        return self.fc(x)
#    def __init__(self):
#        super(Model, self).__init__()
#        self.conv1 = nn.Conv2d(3, 128, kernel_size=10)
#        self.conv2 = nn.Conv2d(128, 256, kernel_size=3)
#        self.conv3 = nn.Conv2d(256, 512, kernel_size=3)
#        self.fc = nn.Linear(512, 10)
        
#    def forward(self, x):
#        batch_size, num_channels, height, width = x.size()
#        hidden = F.relu(self.conv1(x))
#        hidden = F.relu(self.conv2(hidden))
#        hidden = F.relu(self.conv3(hidden))
#        hidden = hidden.view((batch_size, 512, -1))
#        hidden = torch.mean(hidden, dim=-1)
#        logit = self.fc(hidden)
#        return logit
    
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 9, 5)
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(9, 32, 5)
        self.fc1   = nn.Linear(32*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
#class Model(nn.Module):
#    def __init__(self):
#        super(Model, self).__init__()
#        self.hidden_dim = 32*32*3
#        self.conv1 = nn.Conv2d(3, self.hidden_dim, kernel_size=3)
#        self.fc = nn.Linear(self.hidden_dim, 10)
        
#    def forward(self, x):
#        batch_size, num_channels, height, width = x.size()
        
#        hidden = F.relu(self.conv1(x))
#        hidden = hidden.view((batch_size, self.hidden_dim, -1))
#        hidden,_ = torch.max(hidden, dim=-1)
#        logit = self.fc(hidden)
#        return logit
    
    

## Step 3. Defining our training procedure

Set your batch size, learning rate, number of epochs etc. Experiment with various hyper parameters.


In [21]:
# Training settings
batch_size = 64
epochs = 10
lr = .01
momentum = 0.9


train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)


model = Model()
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum) # TODO, Try Adam Optimizer or other optimizers.
optimizer = optim.SGD(model.parameters(), lr=.01, momentum=0.9) # TODO, Try Adam Optimizer or other optimizers.
#optimizer = optim.Adam(model.parameters()) # lr=lr, betas=(0.8, 0.99), eps=1e-08, weight_decay=0, amsgrad=False)

In [22]:

def train_epoch( model, train_loader, optimizer, epoch):
    model.train() # Set the nn.Module to train mode. 
    total_loss = 0
    correct = 0
    num_samples = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        total_loss += loss.detach() # Don't keep computation graph 

    print('Train Epoch: {} \tLoss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            epoch, total_loss / num_samples, 
            correct, 
            num_samples,
            100. * correct / num_samples))


In [23]:
def eval_epoch(model, test_loader, name):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name,
        test_loss, 
        correct, 
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


## Step 4: Training the model

In [24]:

for epoch in range(1, epochs + 1):
    train_epoch(model, train_loader, optimizer, epoch)
    eval_epoch(model,  dev_loader, "Dev")
    print("---")

Train Epoch: 1 	Loss: 0.0285, Accuracy: 12892/40000 (32%)

Dev set: Average loss: 0.0234, Accuracy: 4615/10000 (46%)

---
Train Epoch: 2 	Loss: 0.0215, Accuracy: 20146/40000 (50%)

Dev set: Average loss: 0.0207, Accuracy: 5293/10000 (53%)

---
Train Epoch: 3 	Loss: 0.0188, Accuracy: 22882/40000 (57%)

Dev set: Average loss: 0.0190, Accuracy: 5672/10000 (57%)

---
Train Epoch: 4 	Loss: 0.0168, Accuracy: 24640/40000 (62%)

Dev set: Average loss: 0.0177, Accuracy: 6123/10000 (61%)

---
Train Epoch: 5 	Loss: 0.0154, Accuracy: 26020/40000 (65%)

Dev set: Average loss: 0.0167, Accuracy: 6381/10000 (64%)

---
Train Epoch: 6 	Loss: 0.0141, Accuracy: 27226/40000 (68%)

Dev set: Average loss: 0.0162, Accuracy: 6419/10000 (64%)

---
Train Epoch: 7 	Loss: 0.0131, Accuracy: 27962/40000 (70%)

Dev set: Average loss: 0.0159, Accuracy: 6594/10000 (66%)

---
Train Epoch: 8 	Loss: 0.0122, Accuracy: 28919/40000 (72%)

Dev set: Average loss: 0.0164, Accuracy: 6511/10000 (65%)

---
Train Epoch: 9 	Loss: 0.